## Download data from a .csv file

### Imports

In [1]:
import pandas as pd

The next step is to download the data file from the repository by read_csv()

In [2]:
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/data-science-in-agriculture-basic-statistical-analysis-and-geo-visualisation/estat_aact_eaa01_defaultview_en.csv')

Now let's look at our DataSet.

In [ ]:
df

## Data Preperation

Let's show a list of DataSet colums (pandas.DataFrame.columns)

In [ ]:
df.columns

Let's select only colums 6, 7 and 8 for our future analysis.

In [ ]:
col = df.columns[6:9]  # or [6:-1]
col

Remove all the other columns from the DataSet.

In [ ]:
df = df[col]
df

For further analysis it is necessary to study the general characteristics of the fields using info().

In [ ]:
df.info()

We can see that column 'geo' has an object type. But this column contains country codes. Therefore, it is necessary to change the type of this data to categorical.

In [ ]:
df['geo'] = df['geo'].astype('category')
df.info()

Let's get a list of countries (pandas.DataFrame.unique()).

In [ ]:
df['geo'].unique()

It should be noted that there are some non-standard country codes for the United Kingdom and Greece. We should change the values: UK to GB for the United Kingdom and EL to GR for Greece. To do this, we should add new category names using pandas.Series.cat.add_categories().

In [ ]:
df['geo'] = df['geo'].cat.add_categories(["GB", "GR"])

Then we should change the values using a binary mask:

In [ ]:
pd.options.mode.chained_assignment = None  # swich of the warnings
mask = df['geo'] == 'UK' # Binary mask
df.loc[mask, 'geo'] = "GB" # Change the values for mask
df

Let's do the same for Greece: 'EL'->'GR'

In [ ]:
mask = df['geo'] == "EL"
df.loc[mask, 'geo'] = "GR"
df

After that, add a new column that contains full names of countries. To do this, we can use pycountry library.

In [ ]:
import pycountry

pycountry provides the ISO databases for different standards.

In order to add a column with full country names we need to create a function that will get a country code and return a full name. Then it should be the function pandas.DataFrame.apply() for calculating new column values.

In [ ]:
list_alpha_2 = [i.alpha_2 for i in list(pycountry.countries)]  # create a list of country codes
print("Country codes", list_alpha_2)

def country_flag(df):
    '''
    df: Series
    return: Full name of country or "Invalide code"
    '''
    if (df['geo'] in list_alpha_2):
        return pycountry.countries.get(alpha_2=df['geo']).name
    else:
        print(df['geo'])
        return 'Invalid Code'

df['country_name']=df.apply(country_flag, axis = 1)
df

As you can see, the column with full country names has been added and this DataSet contains a lot of data with an Invalide Code. Let's remove this data using a binary mask.

In [ ]:
mask = df['country_name'] != 'Invalid Code'
df = df[mask]
df

## Statistical Analysis

Let's study this DataSet.

In [ ]:
df.info()

The summary statistics can be calculated easily with the following command: describe().

In [ ]:
df.describe()

As you can see, the result highlights basic statistical information for all the columns except the categorical and object ones. The information includes the total, average, standard deviation, minimum, maximum and the values of the main quarters. In order to display the summary information of category fields, we have to specify the data types we want to display the statistics for:

In [ ]:
df.describe(include=['category'])

As you can see, the statistical information consists of the number of unique values, the value of the most popular category and the number of its values. The detailed information for a specific column can be obtained as follows (value_counts()):

In [ ]:
df['country_name'].value_counts()

You can see that this information is not suitable because the data is not grouped. To get suitable statistics this DataSet should be transformed using a pivot table pivot_table()

In [ ]:
pt_country = pd.pivot_table(df, values= 'OBS_VALUE', index= ['TIME_PERIOD'], columns=['country_name'], aggfunc='sum', margins=True)
pt_country

After that we can calculate statistic description for each country.

In [ ]:
pt_country.describe()

Or we can get statistics for years:

In [ ]:
pt = pd.pivot_table(df, values= 'OBS_VALUE', index= ['country_name'], columns=['TIME_PERIOD'], aggfunc='sum', margins=True)
pt

In [ ]:
pt.describe()

## Data visualization

Now let's visualize our data. We will use Pandas and Matplotlib SeaBorn libraries.

Library documentation

https://matplotlib.org

https://seaborn.pydata.org

Let's build a plot for the last row ('All') except the last values for column ('All'). Pandas inherits Matplotlib function for plotting.

In [ ]:
pt.iloc[-1][:-1].plot()

Let's build a bar plot for summary values for each country (the last column 'All' except the last row).

In [ ]:
pt['All'][:-1].plot.bar(x='country_name', y='val', rot=90)

Let's build a plot for economic accounts dynamics for Sweden.

In [ ]:
pt.loc['Sweden'][:-1].plot()

Let's compare economic accounts for Germany and France on a bar plot. To do this we should make a lot of preparation:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x = np.arange(len(pt.columns)-1)  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots() # Create subplots
rects1 = ax.bar(x - width/2, pt.loc['Germany'][:-1], width, label='Germany') # parameters of bars
rects2 = ax.bar(x + width/2, pt.loc['France'][:-1], width, label='France')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('OBS_VALUE')
ax.set_xlabel('Years')
ax.set_xticks(x)
plt.xticks(rotation = 90)
ax.set_xticklabels(pt.columns[:-1])
ax.legend()

fig.tight_layout()

plt.show()

Also we can build some specific plots using SeaBorn library.

In [ ]:
import seaborn as sns
d = pd.DataFrame(pt.loc['Sweden'][:-1])
d.reset_index(inplace=True)
sns.regplot(x="TIME_PERIOD", y="Sweden", data=d,)

## Build a trend line

Let's make a forecast of dynamics using a linear trend line for Sweden. To build a linear model, it is necessary to create the linear model itself, fit it, test it, and make a prediction. To do this, use sklearn.linear_model.LinearRegression().

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
X=d[['TIME_PERIOD']]
y=d['Sweden']

In [ ]:
model.fit(X, y)

When the model is fitted, we can build our forecast. We should add new values for X and calculate Y.

In [ ]:
X_pred= np.append(X, [2021, 2022, 2023])
X_pred = np.reshape(X_pred, (-1, 1))
# calculate trend
trend = model.predict(X_pred)

plt.plot(X_pred, trend, "-", X, y, ".")

## Interactive maps

### Data transformation for mapping

It is convenient to display the changes of economic accounting on a map to visualize it. There are several libraries for this. It is convenient to use the library plotly.express.

In [ ]:
import plotly.express as px

Let's display our DataSet.

In [ ]:
df

### Download Polygons Of Maps

The next step is to download the map polygons. They are publicly available: https://data.opendatasoft.com/explore/dataset/european-union-countries%40public/information/. Also a DataSet schema is presented on this site. You can see that the key "NAME" of this json is connected to the field "location" in our dataset.

In [ ]:
import json
!wget european-union-countries.geojson "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/data-science-in-agriculture-basic-statistical-analysis-and-geo-visualisation/european-union-countries.geojson"
with open("european-union-countries.geojson", encoding="utf8") as json_file:
    EU_map = json.load(json_file)

The next step is building an interactive map using plotly.express.choropleth(). We should send as input parameters:

Polgons of countries: geojson=EU_map,
Fields for comparison of countries in the DataSet: locations='location',
The key field in the json file that will be compared with locations: featureidkey='properties.name',
The color of countries: color= 'total_cases',
Information for the legend: hover_name= 'location', hover_data= ['location', 'total_cases'],
Animation field: animation_frame= 'Date',
Color scale: color_continuous_scale=px.colors.diverging.RdYlGn[::-1]

Warning: you have to wait a few minutes

In [ ]:
fig = px.choropleth(
    df,
    geojson=EU_map,
    locations='country_name',
    featureidkey='properties.name',    
    color= 'OBS_VALUE', 
    scope='europe',
    hover_name= 'country_name',
    hover_data= ['country_name', 'OBS_VALUE'],
    animation_frame= 'TIME_PERIOD', 
    color_continuous_scale=px.colors.diverging.RdYlGn[::-1]
)

Than we should change some map features. For example: showcountries, showcoastline, showland and fitbouns in function: plotly.express.update_geos(). Also we can modify the map layout: plotly.express.update_layout.

In [ ]:
fig.update_geos(showcountries=False, showcoastlines=False, showland=True, fitbounds=False)

fig.update_layout(
    title_text ="Agriculture Economic accounts",
    title_x = 0.5,
    geo= dict(
        showframe= False,
        showcoastlines= False,
        projection_type = 'equirectangular'
    ),
    margin={"r":0,"t":0,"l":0,"b":0}
)


In [ ]:
from IPython.display import HTML
HTML(fig.to_html())

## Conclusion

As evidenced in practice, the data obtained in real field experiments is not suitable for direct statistical processing. Therefore, in this lab we learned the basic methods of downloading and preliminary data preparation. Unlike the well known classical approaches to statistical data analysis, Python contains many powerful libraries that allow you to manipulate data easily and quickly. Therefore, we have learned the basic methods of automating a library such as Pandas for statistical data analysis. We also learned the basic methods of visualizing the obtained data with the SeaBorn library which also contains effective means of visual data analysis. At the end of the laboratory work, we displayed the DataSet on a dynamic interactive map in * .html format.

## Authors
Yaroslav Vyklyuk, prof., PhD., DrSc

Copyright © 2020 IBM Corporation. This notebook and its source code are released under the terms of the MIT License.